In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.externals import joblib
# from seq_feature_generate import seq_feature_generate
# from seq_feature_generate_v2 import seq_feature_generate_v2
from util_func.eval_function import auroc, cal_ks, auc, ks, fea_psi_calc
#import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import metrics
import sys
import math
from sklearn.metrics import roc_auc_score
import gc
from util_func import TCN_V3 as TCN
from util_func import self_attention as sf
from util_func.look_ahead import Lookahead
from util_func.LayerNormalization import LayerNormalization
import seaborn as sns
import random
from scipy import optimize
from sklearn.metrics import r2_score

load_model =  keras.models.load_model
Sequential = keras.models.Sequential
Dense = keras.layers.Dense
Activation = keras.layers.Activation
Dropout = keras.layers.Dropout
initializers = keras.initializers
regularizers = keras.regularizers
optimizers = keras.optimizers
Input = keras.layers.Input
add = keras.layers.add
Model = keras.models.Model
BatchNormalization = keras.layers.BatchNormalization 
EarlyStopping =  keras.callbacks.EarlyStopping
ModelCheckpoint = keras.callbacks.ModelCheckpoint
K = keras.backend
Concatenate = keras.layers.Concatenate
Reshape = keras.layers.Reshape
Flatten = keras.layers.Flatten
Lambda = keras.layers.Lambda

Using TensorFlow backend.


In [2]:
LOAN_SEQ_LENGTH=10
LOAN_FEATURE_NUM=26

REPAY_SEQ_LENGTH=15
REPAY_FEATURE_NUM=17
# EVENT_ACTION_SEQ_LENGTH=30
# EVENT_ACTION_FEATURE_NUM=2

EVENT_SEQ_LENGTH=20
EVENT_FEATURE_NUM=55

STAT_FEATURE_NUM = 194

In [3]:
data_dir = '/home/odin/chengbixiao/git_code/User_loan_demand/user_loan_demand/offline/data/'

train_info_path = data_dir + 'feature_model_file/train_info.csv'
test1_info_path = data_dir + 'feature_model_file/test1_info.csv'
test_info_path = data_dir + 'feature_model_file/test_info.csv'

train_label_path = data_dir + 'feature_model_file/train_label.npy'
test1_label_path = data_dir + 'feature_model_file/test1_label.npy'
test_label_path = data_dir + 'feature_model_file/test_label.npy'

train_loan_seq_matrix_path = data_dir + 'feature_model_file/train_loan_seq_matrix.npy'
train_repay_seq_matrix_path = data_dir + 'feature_model_file/train_repay_seq_matrix.npy'
train_event1_seq_matrix_path = data_dir + 'feature_model_file/train_event1_seq_matrix.npy'
train_event2_seq_matrix_path = data_dir + 'feature_model_file/train_event2_seq_matrix.npy'

test1_loan_seq_matrix_path = data_dir + 'feature_model_file/test1_loan_seq_matrix.npy'
test1_repay_seq_matrix_path = data_dir + 'feature_model_file/test1_repay_seq_matrix.npy'
test1_event1_seq_matrix_path = data_dir + 'feature_model_file/test1_event1_seq_matrix.npy'
test1_event2_seq_matrix_path = data_dir + 'feature_model_file/test1_event2_seq_matrix.npy'

test_loan_seq_matrix_path = data_dir + 'feature_model_file/test_loan_seq_matrix.npy'
test_repay_seq_matrix_path = data_dir + 'feature_model_file/test_repay_seq_matrix.npy'
test_event1_seq_matrix_path = data_dir + 'feature_model_file/test_event1_seq_matrix.npy'
test_event2_seq_matrix_path = data_dir + 'feature_model_file/test_event2_seq_matrix.npy'

stat_feature_train_norm_path = data_dir + 'feature_model_file/stat_feature_train.npy'
stat_feature_test1_norm_path = data_dir + 'feature_model_file/stat_feature_test1.npy'
stat_feature_test_norm_path = data_dir + 'feature_model_file/stat_feature_test.npy'

save_model_file = data_dir + 'model_file/user_demand_model_tmp_0820.h5'
save_pb_file_path = data_dir + 'model_file'
save_pb_file_name = 'user_demand_model_online.pb'

new_model_pred_train_path = data_dir + 'model_file/new_model_pred_train.csv'
new_model_pred_test_path = data_dir + 'model_file/new_model_pred_test.csv'
new_model_pred_test1_path = data_dir + 'model_file/new_model_pred_test1.csv'

wy_eval_path = data_dir + 'model_file/cali_score_final_eval.csv'

In [4]:
train_event_seq_matrix = np.load(train_event2_seq_matrix_path)
train_loan_seq_matrix = np.load(train_loan_seq_matrix_path)
train_repay_seq_matrix = np.load(train_repay_seq_matrix_path)

test_event_seq_matrix = np.load(test_event2_seq_matrix_path)
test_loan_seq_matrix = np.load(test_loan_seq_matrix_path)
test_repay_seq_matrix = np.load(test_repay_seq_matrix_path)

test1_event_seq_matrix = np.load(test1_event2_seq_matrix_path)
test1_loan_seq_matrix = np.load(test1_loan_seq_matrix_path)
test1_repay_seq_matrix = np.load(test1_repay_seq_matrix_path)

baseline_feature_top_train_norm = np.load(stat_feature_train_norm_path)
baseline_feature_top_test_norm = np.load(stat_feature_test_norm_path)
baseline_feature_top_test1_norm = np.load(stat_feature_test1_norm_path)

train_label = np.load(train_label_path)
test_label = np.load(test_label_path)
test1_label = np.load(test1_label_path)

In [5]:
train_event_seq_matrix.shape, train_loan_seq_matrix.shape, train_repay_seq_matrix.shape

((781990, 20, 55), (781990, 10, 26), (781990, 15, 17))

In [8]:
def Demand_model_attention_structure():
   
    K.clear_session()

    # --------------------------------------------- encoders ------------------------------------
    # ---------- loan seq encoder ----------
    loan_inputs = keras.layers.Input(shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM), name='loan_seq_input')
    print('use gru to encode loan seq...')
    # use gru to encode loan seq
    l1_param, l2_param=1e-5, 1e-5
    loan_o = keras.layers.GRU( 
        input_shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM),
        units=32, use_bias=True, 
        return_sequences=False,
        kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
        name = 'loan_gru')(loan_inputs)

    
    # -------------- repay seq encoder -------------
    repay_inputs = keras.layers.Input(shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM), name='repay_seq_input')
    print('use gru to encode repay seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    repay_o = keras.layers.GRU( 
                input_shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM),
                units=32, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='repay_gru')(repay_inputs)

    
    # -------------- event seq encoder ---------------
    event_inputs = keras.layers.Input(shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM), name='event_seq_input')
    print('use gru to encode event seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    event_o = keras.layers.GRU( 
                input_shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM),
                units=32, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='event_gru')(event_inputs)
    print(event_o)
    
    
    # self attention for click, loan, repay encode vector
    combine_input_multi = keras.layers.concatenate([loan_o, event_o], name='encoder_concatenate')
    combine_input_multi = keras.layers.Reshape((2,32), name = 'reshape_1')(combine_input_multi)
    seq_encoder = sf.Self_Attention_cross_seq(32,32)(combine_input_multi)
    seq_encoder = keras.layers.Flatten(name='flatten_encoder')(seq_encoder)

    # --------- baseline dense layer ---------
    baseline_input = keras.layers.Input(shape=(STAT_FEATURE_NUM,), name='baseline_top_inputs')

    baseline_o = Dense(128, activation = 'relu',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=1e-6),name = 'layer1')(baseline_input)
    baseline_o = Dropout(0.2)(baseline_o)

    baseline_o = Dense(64, activation = 'relu',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=1e-6),name = 'layer2')(baseline_o)
    baseline_o = Dropout(0.1)(baseline_o)
    
    
    # -------- combine layer ------------
    combine_o = keras.layers.concatenate([baseline_o, seq_encoder], name='comine_input')

    # -------- combine dense ------------
    o = Dense(1, activation = 'sigmoid',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=0.0),name = 'fc_output')(combine_o)

    # whole global model
    combine_model = Model(inputs = [loan_inputs, repay_inputs, event_inputs, baseline_input], outputs = o)
    print(combine_model.summary())

    return combine_model


In [5]:
def Demand_model_structure():
   
    K.clear_session()

    # --------------------------------------------- encoders ------------------------------------
    # ---------- loan seq encoder ----------
    loan_inputs = keras.layers.Input(shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM), name='loan_seq_input')
    print('use gru to encode loan seq...')
    # use gru to encode loan seq
    l1_param, l2_param=1e-5, 1e-5
    loan_o = keras.layers.GRU( 
        input_shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM),
        units=26, use_bias=True, 
        return_sequences=False,
        kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
        name = 'loan_gru')(loan_inputs)
    print(loan_o)

    

    # -------------- repay seq encoder -------------
    repay_inputs = keras.layers.Input(shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM), name='repay_seq_input')
    print('use gru to encode repay seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    repay_o = keras.layers.GRU( 
                input_shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM),
                units=17, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='repay_gru')(repay_inputs)
    print(repay_o)
    
    # -------------- event seq encoder ---------------
#     event_inputs = keras.layers.Input(shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM), name='click_seq_input')
#     print('use tcn to encode event seq...')
#     # use tcn to encode click seq
#     event_tcn = TCN.TCN(nb_filters=32,
#                 kernel_size=7,
#                 nb_stacks=1,
#                 dilations=(1, 7),
#                 padding='causal',
#                 use_skip_connections=False,
#                 dropout_rate=0.3,
#                 return_sequences=True,
#                 activation='relu',
#                 name='event_tcn',
#                 kernel_initializer='he_normal',
#                 use_batch_norm=False,
#                 block_size=1)(event_inputs)

#     event_o = Lambda(lambda tt: keras.layers.average([tt[:,EVENT_SEQ_LENGTH//2,:], tt[:, -1, :]]))(event_tcn)
#     print(event_o)
    
    event_inputs = keras.layers.Input(shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM), name='event_seq_input')
    print('use gru to encode event seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    event_o = keras.layers.GRU( 
                input_shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM),
                units=55, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='event_gru')(event_inputs)
    print(event_o)
    
    
    # self attention for click, loan, repay encode vector
    combine_input_multi = keras.layers.concatenate([loan_o, repay_o, event_o], name='encoder_concatenate')
#     combine_input_multi = keras.layers.Reshape((3,32), name = 'reshape_1')(combine_input_multi)
#     seq_encoder = sf.Self_Attention_cross_seq(32,32)(combine_input_multi)
#     seq_encoder = keras.layers.Flatten(name='flatten_encoder')(seq_encoder)

    # --------- baseline dense layer ---------
    baseline_input = keras.layers.Input(shape=(197,), name='baseline_top_inputs')

    baseline_o = Dense(128, activation = 'relu',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=1e-6),name = 'layer1')(baseline_input)
    baseline_o = Dropout(0.2)(baseline_o)

    baseline_o = Dense(64, activation = 'relu',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=1e-6),name = 'layer2')(baseline_o)
    baseline_o = Dropout(0.1)(baseline_o)
    
    
    # -------- combine layer ------------
    combine_o = keras.layers.concatenate([baseline_o, combine_input_multi], name='comine_input')

    # -------- combine dense ------------
    o = Dense(1, activation = 'sigmoid',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=0.0, l1=0.0),name = 'fc_output')(combine_o)

    # whole global model
    combine_model = Model(inputs = [loan_inputs, repay_inputs, event_inputs, baseline_input], outputs = o)
    print(combine_model.summary())

    return combine_model


In [6]:
def Demand_model_structure_only_seq():
   
    K.clear_session()

    # --------------------------------------------- encoders ------------------------------------
    # ---------- loan seq encoder ----------
    loan_inputs = keras.layers.Input(shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM), name='loan_seq_input')
    print('use gru to encode loan seq...')
    # use gru to encode loan seq
    l1_param, l2_param=1e-5, 1e-5
    loan_o = keras.layers.GRU( 
        input_shape=(LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM),
        units=26, use_bias=True, 
        return_sequences=False,
        kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
        activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
        name = 'loan_gru')(loan_inputs)
    print(loan_o)

    # -------------- repay seq encoder -------------
    repay_inputs = keras.layers.Input(shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM), name='repay_seq_input')
    print('use gru to encode repay seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    repay_o = keras.layers.GRU( 
                input_shape=(REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM),
                units=17, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='repay_gru')(repay_inputs)
    print(repay_o)
    
    # -------------- click seq encoder ---------------
#     event_inputs = keras.layers.Input(shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM), name='click_seq_input')
#     print('use tcn to encode event seq...')
#     # use tcn to encode click seq
#     event_tcn = TCN.TCN(nb_filters=32,
#                 kernel_size=7,
#                 nb_stacks=1,
#                 dilations=(1, 7),
#                 padding='causal',
#                 use_skip_connections=False,
#                 dropout_rate=0.3,
#                 return_sequences=True,
#                 activation='relu',
#                 name='event_tcn',
#                 kernel_initializer='he_normal',
#                 use_batch_norm=False,
#                 block_size=1)(event_inputs)

#     event_o = Lambda(lambda tt: keras.layers.average([tt[:,EVENT_SEQ_LENGTH//2,:], tt[:, -1, :]]))(event_tcn)
#     print(event_o)
    
    event_inputs = keras.layers.Input(shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM), name='event_seq_input')
    print('use gru to encode event seq...')
    # use gru to encode repay seq
    l1_param, l2_param=1e-5, 1e-5
    event_o = keras.layers.GRU( 
                input_shape=(EVENT_SEQ_LENGTH, EVENT_FEATURE_NUM),
                units=55, use_bias=True, 
                return_sequences=False,
                kernel_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                recurrent_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                bias_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param), 
                activity_regularizer=keras.regularizers.l1_l2(l1=l1_param, l2=l2_param),
                name='event_gru')(event_inputs)
    print(event_o)
    
    
    # self attention for click, loan, repay encode vector
    combine_input_multi = keras.layers.concatenate([loan_o, repay_o, event_o], name='encoder_concatenate')
#     combine_input_multi = keras.layers.Reshape((3,32), name = 'reshape_1')(combine_input_multi)
#     seq_encoder = sf.Self_Attention_cross_seq(32,32)(combine_input_multi)
#     seq_encoder = keras.layers.Flatten(name='flatten_encoder')(seq_encoder)

    # -------- combine dense ------------
    o = Dense(1, activation = 'sigmoid',
            kernel_initializer = keras.initializers.random_normal(mean=0.0, stddev=0.05, seed=None),
            bias_initializer = keras.initializers.constant(value=0),
            kernel_regularizer= keras.regularizers.l1_l2(l2=1e-4, l1=0.0),name = 'fc_output')(combine_input_multi)

    # whole global model
    combine_model = Model(inputs = [loan_inputs, repay_inputs,event_inputs], outputs = o)

    print(combine_model.summary())

    return combine_model

In [11]:
def update_model_refresh(save_model_file, 
                         train_loan_seq_matrix, train_repay_seq_matrix,train_event_seq_matrix, 
                         baseline_feature_top_train_norm,train_label, 
                         test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix, 
                         baseline_feature_top_test_norm,test_label):
    # Setting optimization config
#     combine_model_1=Demand_model_structure()
#     combine_model_1=Demand_model_structure_only_seq()
    combine_model_1=Demand_model_attention_structure()
    
    learning_rate= 1e-3
    adam = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07,)
    look_ahead = Lookahead(adam, sync_period=5, slow_step=0.5)
    
    combine_model_1.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = [auroc], sample_weight_mode = None) 
    
    # # Training
    early_stopping = EarlyStopping(monitor='val_auroc', patience=20, verbose=2, mode='max')
    filepath=save_model_file
    checkpoint = ModelCheckpoint(filepath, monitor='val_auroc', verbose=1, save_best_only=True, mode='max')
    
    history_v1 = combine_model_1.fit(
        
        [train_loan_seq_matrix, train_repay_seq_matrix, train_event_seq_matrix, baseline_feature_top_train_norm],
#         [train_loan_seq_matrix, train_repay_seq_matrix, train_event_seq_matrix],

        train_label,
        epochs = 50,
        batch_size = 512,
        
        #verbose = 10,  ## 每个epoch输出一行记录
        # validation_split = 0.25,
        
        validation_data = ([test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix,baseline_feature_top_test_norm], test_label),
#         validation_data = ([test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix], test_label),

        shuffle = True,
        class_weight = {0:1, 1:1},
        initial_epoch=0,
        callbacks=[early_stopping, checkpoint]
    )      
    print('done!')

In [8]:
def model_evaluation_compare(new_model_filepath,
                            train_event_seq_matrix, train_loan_seq_matrix, train_repay_seq_matrix, baseline_feature_top_train_norm,train_label,
                            test1_event_seq_matrix, test1_loan_seq_matrix, test1_repay_seq_matrix, baseline_feature_top_test1_norm,test1_label,
                            test_event_seq_matrix, test_loan_seq_matrix, test_repay_seq_matrix, baseline_feature_top_test_norm,test_label,
                            new_model_pred_train_path,new_model_pred_test_path,new_model_pred_test1_path
                            ):
    ### evaluation new_model
    model = Demand_model_structure()
    model.load_weights(new_model_filepath)
    print("-------------------model evaluatie------------------")
    print("evaluate on train data:")
    pred_train = model.predict([train_loan_seq_matrix, train_repay_seq_matrix,train_event_seq_matrix, baseline_feature_top_train_norm])
    train_auc = auc(pred_train[:,0], train_label)
    train_ks = ks(train_label, pred_train[:,0])
    print(train_auc, train_ks)

    print("evaluate on test data:")
    pred_test = model.predict([test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix, baseline_feature_top_test_norm])
    test_auc = auc(pred_test[:,0], test_label)
    test_ks = ks(test_label, pred_test[:,0])
    print(test_auc, test_ks)

    print("evaluate on test data:")
    pred_test1 = model.predict([test1_loan_seq_matrix, test1_repay_seq_matrix, test1_event_seq_matrix, baseline_feature_top_test1_norm])
    test1_auc = auc(pred_test1[:,0], test1_label)
    test1_ks = ks(test1_label, pred_test1[:,0])
    print(test_auc, test_ks)

    # output 
    pd.DataFrame(pred_train,columns=['new_model_pred_train']).to_csv(new_model_pred_train_path, sep='\t',index=0)
    pd.DataFrame(pred_test,columns=['new_model_pred_test']).to_csv(new_model_pred_test_path, sep='\t',index=0)
    pd.DataFrame(pred_test1,columns=['new_model_pred_test1']).to_csv(new_model_pred_test1_path, sep='\t',index=0)

    ### compare new_model
    model.load_weights(new_model_filepath)

    print("evaluate on train data:")
    new_model_pred_train = model.predict([train_loan_seq_matrix, train_repay_seq_matrix, train_event_seq_matrix, baseline_feature_top_train_norm])
    train_auc = auc(new_model_pred_train[:,0], train_label)
    train_ks = ks(train_label, new_model_pred_train[:,0])
    print(train_auc, train_ks)

    print("evaluate on test data:")
    new_model_pred_test = model.predict([test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix, baseline_feature_top_test_norm])
    test_auc = auc(new_model_pred_test[:,0], test_label)
    test_ks = ks(test_label, new_model_pred_test[:,0])
    print(test_auc, test_ks)

    print("evaluate on test1 data:")
    new_model_pred_test1 = model.predict([test1_loan_seq_matrix, test1_repay_seq_matrix, test1_event_seq_matrix, baseline_feature_top_test1_norm])
    test1_auc = auc(new_model_pred_test1[:,0], test1_label)
    test1_ks = ks(test1_label, new_model_pred_test1[:,0])
    print(test1_auc, test1_ks)

    pd.DataFrame(new_model_pred_train,columns=['new_model_pred_train']).to_csv(new_model_pred_train_path, sep='\t',index=0)
    pd.DataFrame(new_model_pred_test,columns=['new_model_pred_test']).to_csv(new_model_pred_test_path, sep='\t',index=0)
    pd.DataFrame(new_model_pred_test1,columns=['new_model_pred_test1']).to_csv(new_model_pred_test1_path, sep='\t',index=0)
    print('done!')

In [12]:
update_model_refresh('./baseline2_nn_attention_0902.h5',
                     train_loan_seq_matrix, train_repay_seq_matrix, train_event_seq_matrix, 
                     baseline_feature_top_train_norm,train_label, 
                     test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix, 
                     baseline_feature_top_test_norm,test_label)

use gru to encode loan seq...
use gru to encode repay seq...
use gru to encode event seq...
Tensor("event_gru/TensorArrayReadV3:0", shape=(?, 32), dtype=float32)
WQ.shape (?, 2, 32)
K.permute_dimensions(WK, [0, 2, 1]).shape (?, 32, 2)
QK.shape (?, 2, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
loan_seq_input (InputLayer)     (None, 10, 26)       0                                            
__________________________________________________________________________________________________
event_seq_input (InputLayer)    (None, 20, 55)       0                                            
__________________________________________________________________________________________________
baseline_top_inputs (InputLayer (None, 194)          0                                            
_____________________________________________________

In [13]:
combine_model=Demand_model_attention_structure()

use gru to encode loan seq...
use gru to encode repay seq...
use gru to encode event seq...
Tensor("event_gru/TensorArrayReadV3:0", shape=(?, 32), dtype=float32)
WQ.shape (?, 2, 32)
K.permute_dimensions(WK, [0, 2, 1]).shape (?, 32, 2)
QK.shape (?, 2, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
loan_seq_input (InputLayer)     (None, 10, 26)       0                                            
__________________________________________________________________________________________________
event_seq_input (InputLayer)    (None, 20, 55)       0                                            
__________________________________________________________________________________________________
baseline_top_inputs (InputLayer (None, 194)          0                                            
_____________________________________________________

In [9]:
combine_model = Demand_model_attention_structure()
combine_model.load_weights('./baseline2_nn_attention_0902.h5')

print("-------------------model evaluatie------------------")
print("evaluate on train data:")
pred_train = combine_model.predict([train_loan_seq_matrix, train_repay_seq_matrix, train_event_seq_matrix, baseline_feature_top_train_norm])
train_auc = auc(pred_train[:,0], train_label)
train_ks = ks(train_label, pred_train[:,0])
print(train_auc, train_ks)

print("evaluate on test data:")
pred_test = combine_model.predict([test_loan_seq_matrix, test_repay_seq_matrix, test_event_seq_matrix,baseline_feature_top_test_norm])
test_auc = auc(pred_test[:,0], test_label)
test_ks = ks(test_label, pred_test[:,0])
print(test_auc, test_ks)

print("evaluate on test1 data:")
pred_test1 = combine_model.predict([test1_loan_seq_matrix, test1_repay_seq_matrix, test1_event_seq_matrix,baseline_feature_top_test1_norm])
test1_auc = auc(pred_test1[:,0], test1_label)
test1_ks = ks(test1_label, pred_test1[:,0])
print(test1_auc, test1_ks)

use gru to encode loan seq...
Instructions for updating:
Colocations handled automatically by placer.
use gru to encode repay seq...
use gru to encode event seq...
Tensor("event_gru/TensorArrayReadV3:0", shape=(?, 32), dtype=float32)
WQ.shape (?, 2, 32)
K.permute_dimensions(WK, [0, 2, 1]).shape (?, 32, 2)
QK.shape (?, 2, 2)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
loan_seq_input (InputLayer)     (None, 10, 26)       0                                            
__________________________________________________________________________________________________
event_seq_input (InputLayer)    (None, 20, 55)       0                                            
________________________________________________________________

In [12]:
plot_model(combine_model_1, 'model2.png', show_shapes=True)

In [11]:
from keras.utils import plot_model